# Hopfield Network

## Introduction

### Structure
![Structure](Hopfield-net.png)

### Updating
Here we update the network asynchronously by randomly choosing one node and update its ouput.
For a network with $N$ nodes:
$$s_{i}(t+1)=sgn(\sum_{j=1}^{N}w_{ij}s_{j}(t)-\theta_{i})$$
$$sgn(x)=1 (x\geq0)$$
$$sgn(x)=-1(otherwise)$$
$$W=W^{T}$$
$$W_{ii}>=0$$
Where $w_{ij}$ indicates the entry at $i_{th}$ row and $j_{th}$ column of $W$.
asynchronous
### Energy
#### Definition
$$E=-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}w_{ij}s_{i}s_{j}+\sum_{i=1}^{N}\theta_{i}s_{i}$$
or in matrix form:
$$E=-\frac{1}{2}s^{T}Ws+\theta^{T}s$$
where:
$$s=\begin{bmatrix}s_{1}&s_{2}&\cdots&s_{N}\end{bmatrix}^{T}$$
$$\theta=\begin{bmatrix}\theta_{1}&\theta_{2}&\cdots&\theta_{N}\end{bmatrix}^{T}$$
#### Proof
* E is bounded
$$\lvert E\rvert\leq\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}\lvert w_{ij}\rvert \lvert s_{i}\rvert\lvert s_{j}\rvert+\sum_{i=1}^{N}\lvert\theta_{i}\rvert\lvert s_{i}\rvert$$
* E is monotonically decreasing

Assume that the $k_{th}$ node has been updated:
$$s(t+1)=s(t)+\Delta s(t)$$
$$\Delta s(t)=\epsilon_{k}\Delta s_{k}(t)$$
\begin{equation}
\begin{split}
\Delta E&=E(t+1)-E(t)\\
&=-\frac{1}{2}\left(\left(s(t)+\Delta s(t)\right)^{T}W\left(s(t)+\Delta s(t)\right)\right)+\theta^{T}\left(s(t)+\Delta s(t)\right)\\
&-\left(-\frac{1}{2}s(t)^{T}Ws(t)+\theta^{T}s(t)\right)\\
&=-\frac{1}{2}\left(\Delta s(t)^{T}Ws(t)+ s(t)^{T}W\Delta s(t)+\Delta s(t)^{T}W\Delta s(t)\right)+\theta^{T}\Delta s(t)\\
&=-\frac{1}{2}\left[\Delta s_{k}(t)\sum_{j=1}^{N}w_{kj}s_{j}(t)+\Delta s_{k}(t)\sum_{i=1}^{N}w_{ik}s_{i}(t)+w_{kk}\Delta s_{k}^{2}(t)\right]+\Delta s_{k}(t)\theta_{k}
\end{split}
\end{equation}
where $\epsilon_{k}$ denotes the vector with $k_{th}$ entry equal to 1.
Given that $w_{ij}=w_{ji}$,
$$\Delta E=-\Delta s_{k}(t)\left(\sum_{j=1}^{N}w_{kj}s_{j}(t)-\theta_{k}\right)-\frac{1}{2}w_{kk}\Delta s_{k}^{2}(t)
$$
Note that$\Delta s_{k}(t)\left(\sum_{j=1}^{N}w_{kj}s_{j}(t)-\theta_{k}\right)\geq0$ and that $w_{kk}\geq0$, we obtain:
$$\Delta E\leq-\frac{1}{2}w_{kk}\Delta s_{k}^{2}(t)\leq 0$$
Hence, E is monotonically decreasing. 

Thus, we have the conclusion that the energy of the network will converge to a local minimum after several iterations. 

### A numerical example for convergence

In [1]:
import numpy as np
import random

In [2]:
def updateState(state,W,theta,verbose=False):
    networkSize=np.shape(state)[0]
    if np.shape(state)!=np.shape(theta):
        raise(ValueError("The shapes of states and thresholds didn't match."))
    if networkSize!=np.shape(W)[0]:
        raise(ValueError("The shapes of states and W didn't match."))
    if np.shape(W)[0]!=np.shape(W)[1]:
        raise(ValueError("W must be a square matrix."))
    lastState=None
    iteration=0
    n=0
    k=random.randint(0,networkSize-1)
    if verbose:
        print("Original state:\n{}".format(state))
    while n<networkSize:
        iteration+=1
        #lastState=np.array(state,copy=True)
        lastState=np.copy(state)
        if np.sum(W[k,:]*state)-theta[k]>0:
            state[k]=1
        elif np.sum(W[k,:]*state)-theta[k]<0:
            state[k]=-1
        else:
            state[k]=0
        if type(lastState)==type(None) or np.any(lastState!=state):
            n=0
        else:
            n+=1
        if verbose:
            print("Node number {} has been updated at epoch {}.".format(k,iteration))
            print("New state:\n{}".format(state))
        if k<networkSize-1:
            k+=1
        else:
            k=0

In [3]:
initState=np.array([1,-1,-1,-1])
W=np.array([[0,2,2,2],[2,0,2,2],[2,2,0,2],[2,2,2,0]])
theta=np.array([1,2,3,4])
updateState(initState,W,theta,verbose=True)

Original state:
[ 1 -1 -1 -1]
Node number 3 has been updated at epoch 1.
New state:
[ 1 -1 -1 -1]
Node number 0 has been updated at epoch 2.
New state:
[-1 -1 -1 -1]
Node number 1 has been updated at epoch 3.
New state:
[-1 -1 -1 -1]
Node number 2 has been updated at epoch 4.
New state:
[-1 -1 -1 -1]
Node number 3 has been updated at epoch 5.
New state:
[-1 -1 -1 -1]
Node number 0 has been updated at epoch 6.
New state:
[-1 -1 -1 -1]


In [4]:
initState=np.array([1,1,1,-1])
updateState(initState,W,theta,verbose=True)

Original state:
[ 1  1  1 -1]
Node number 0 has been updated at epoch 1.
New state:
[ 1  1  1 -1]
Node number 1 has been updated at epoch 2.
New state:
[ 1  0  1 -1]
Node number 2 has been updated at epoch 3.
New state:
[ 1  0 -1 -1]
Node number 3 has been updated at epoch 4.
New state:
[ 1  0 -1 -1]
Node number 0 has been updated at epoch 5.
New state:
[-1  0 -1 -1]
Node number 1 has been updated at epoch 6.
New state:
[-1 -1 -1 -1]
Node number 2 has been updated at epoch 7.
New state:
[-1 -1 -1 -1]
Node number 3 has been updated at epoch 8.
New state:
[-1 -1 -1 -1]
Node number 0 has been updated at epoch 9.
New state:
[-1 -1 -1 -1]
Node number 1 has been updated at epoch 10.
New state:
[-1 -1 -1 -1]


### Design $W$
#### Hebb
#### Pseudo Inverse
Given $M$ linear independent samples $\begin{bmatrix}X_{1}&X_{2}&\cdots&X_{M}\end{bmatrix}$